## View  RubinLSST cutout images and Lupton colored image

- author : Sylvie Dagoret-Campagne
- creation date : 2025-05-30
- last update : 2025-05-30 

In [ ]:
import astroquery
print(astroquery.__version__)

https://stackoverflow.com/questions/59699193/how-to-obtain-2d-cutout-of-an-image-from-a-skycoord-position

In [ ]:
# For Angle conversion
import matplotlib.pyplot as plt
from astropy.visualization.wcsaxes import WCSAxes
from astropy.coordinates import Angle
import astropy.units as u
from astropy.coordinates import SkyCoord
import pandas as pd
import numpy as np
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy.nddata import Cutout2D
from astropy.visualization import simple_norm, ZScaleInterval,PercentileInterval,make_lupton_rgb
from itertools import combinations

In [ ]:
plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams['axes.titlesize']= 'xx-large'

In [ ]:
from IPython.display import Image, display

In [ ]:
import os,re

In [ ]:
def load_fits_image_with_norm(path):
    """
    Load rubin cutout adapted to rubin LSST
    """
    with fits.open(path) as hdul:
        # where is the image
        data = hdul[1].data
        if data is None:
            raise ValueError(f"No image data in {path}")
        # where is the header that contains WCS
        wcs = WCS(hdul[0].header)
        
    vmin, vmax = ZScaleInterval().get_limits(data)
    norm = simple_norm(data, stretch='asinh', vmin=vmin, vmax=vmax)
    return data, wcs, norm

In [ ]:
def make_lupton_image(data_b, data_g, data_r, stretch=0.5, Q=10):
    return make_lupton_rgb(image_r=data_r, image_g=data_g, image_b=data_b,
                           stretch=stretch, Q=Q)

In [ ]:
def compute_color_difference(data_band1, data_band2):
    """
    Calcule la différence de magnitudes entre deux bandes : -2.5 * log10(b1) + 2.5 * log10(b2)
    """
    with np.errstate(divide='ignore', invalid='ignore'):
        mask = (data_band1 > 0) & (data_band2 > 0)
        diff = np.full_like(data_band1, np.nan)
        diff[mask] = -2.5 * np.log10(data_band1[mask]) + 2.5 * np.log10(data_band2[mask])
    return diff


In [ ]:
def compute_common_limits(diff_maps, quantile_clip=0.01):
    all_values = np.concatenate([d[np.isfinite(d)].ravel() for d in diff_maps])
    vmin = np.quantile(all_values, quantile_clip)
    vmax = np.quantile(all_values, 1 - quantile_clip)
    return vmin, vmax

## Configuration

### Target in ECDFS

- path to article : https://arxiv.org/pdf/1104.0931
- visual selection of the tiles : https://archive.stsci.edu/prepds/gems/browser.html
- path file download : https://archive.stsci.edu/pub/hlsp/gems/v_mk1/

In [ ]:
fig_table1 = "input_figs/table1_gemcandidates.png"
fig_table2 = "input_figs/table2_gemcandidates.png"

In [ ]:
Image(url= fig_table1,width=800)

In [ ]:
Image(url= fig_table2,width=500)

In [ ]:
#15422 44 03:32:38.21 –27:56:53.2 
ra1 = "03:32:38.21 hours"
dec1 = "-27:56:53.2 degrees"
tile1 = 44

#34244 94 03:32:06.45 –27:47:28.6 
ra2 = "03:32:06.45 hours"
dec2 = "-27:47:28.6 degrees"
tile2 = 94
# Je ne trouve pas cette tile ==> FindTileForCutoutGEM
tile2 = 32

#40173 35 03:33:19.45 –27:44:50.0 
ra3 = "03:33:19.45 hours"
dec3 = "-27:44:50.0 degrees"
tile3 = 35

#43242 45 03:31:55.35 –27:43:23.5 
ra4 = "03:31:55.35 hours"
dec4 = "-27:43:23.5 degrees"
tile4 = 45

#46446 47 03:31:35.94 –27:41:48.2 
ra5 = "03:31:35.94 hours"
dec5 = "-27:41:48.2 degrees"
tile5 = 47

#12589 03:31:24.89 −27:58:07.0
ra6 = "03:31:24.89 hours"
dec6 = "-27:58:07.0 degrees"
tile6 = 17

#43797 03:31:31.74 −27:43:00.8 
ra7 = "03:31:31.74 hours"
dec7 = "-27:43:00.8 degrees"
tile7 = 47

#28294 03:31:50.54 −27:50:28.4 
ra8 = "03:31:50.54 hours"
dec8 = "-27:50:28.4 degrees"
tile8 = 33

#36857 03:31:53.24 −27:46:18.9
ra9 = "03:31:53.24 hours"
dec9 = "-27:46:18.9 degrees"
tile9 = 38

#36714 03:32:59.78 −27:46:26.4 
ra10 = "03:32:59.78 hours"
dec10 = "-27:46:26.4 degrees"
tile10 = 37


In [ ]:
lsstcomcam_targets = {}
# high rank
lsstcomcam_targets["ECDFS_G15422"] = {"field_name": "GEMS-15422", "ra": 53.159208333333325, "dec": -27.94811111111111,"tile":tile1}
lsstcomcam_targets["ECDFS_G34244"] = {"field_name": "GEMS-34244", "ra": 53.02687499999999 , "dec": -27.79127777777778,"tile":tile2}
lsstcomcam_targets["ECDFS_G40173"] = {"field_name": "GEMS-40173", "ra": 53.33104166666666 , "dec": -27.747222222222224,"tile":tile3}
lsstcomcam_targets["ECDFS_G43242"] = {"field_name": "GEMS-43242", "ra": 52.980624999999996 , "dec": -27.72319444444444,"tile":tile4}
lsstcomcam_targets["ECDFS_G46446"] = {"field_name": "GEMS-46446", "ra": 52.89975 , "dec": -27.696722222222224,"tile":tile5}

# low rank
lsstcomcam_targets["ECDFS_G12589"] = {"field_name": "GEMS-12589", "ra": 52.85370833333333, "dec": -27.96861111111111,"tile":tile6}
lsstcomcam_targets["ECDFS_G43797"] = {"field_name": "GEMS-43797", "ra": 52.88224999999999, "dec": -27.71688888888889,"tile":tile7}

lsstcomcam_targets["ECDFS_G28294"] = {"field_name": "GEMS-28294", "ra": 52.960583333333325 , "dec": -27.84122222222222,"tile":tile8}
lsstcomcam_targets["ECDFS_G6857"] = {"field_name": "GEMS-6857", "ra": 52.97183333333333 , "dec": -27.771916666666666,"tile":tile9}
lsstcomcam_targets["ECDFS_G36714"] = {"field_name": "GEMS-36714", "ra": 53.249083333333324, "dec": -27.773999999999997,"tile":tile10}


In [ ]:
df = pd.DataFrame(lsstcomcam_targets).T

In [ ]:
# candidates
key = "ECDFS_G15422"
#key = "ECDFS_G34244"
#key = "ECDFS_G40173"
#key= "ECDFS_G43242"
#key= "ECDFS_G46446"

# unknown
#key = "ECDFS_G12589"
#key = "ECDFS_G43797"
#key = "ECDFS_G28294"
#key = "ECDFS_G6857"
#key = "ECDFS_G36714"

the_target = lsstcomcam_targets[key]
target_ra = the_target["ra"]
target_dec = the_target["dec"]
target_name = the_target["field_name"]
tile_num = the_target["tile"]
coord = SkyCoord(ra=target_ra, dec=target_dec, unit=(u.deg, u.deg))

target_title = (
    the_target["field_name"] + f" (ra,dec) = ({target_ra:.2f},{target_dec:.2f}) "
)

list_of_keys = ["ECDFS_G15422", "ECDFS_G34244","ECDFS_G40173","ECDFS_G43242","ECDFS_G46446","ECDFS_G12589","ECDFS_G43797","ECDFS_G28294","ECDFS_G6857","ECDFS_G36714"]

In [ ]:
subpath_img = re.findall("^ECDFS_(.*)",key)[0]
subpath_img

## Configuration

In [ ]:
all_bands = ["u", "g", "r", "i", "z", "y"]
all_bands_colors = ["blue", "green", "red", "orange", "yellow", "purple"]
all_bands_cmaps = ["Blues","Greens","Reds","Oranges","Greys","Purples"]
NB = len(all_bands)

In [ ]:
toppath_rubincutout = "../2025-05-24-SL-ECDFS-Fit/data"
path_rubincutout = os.path.join(toppath_rubincutout,subpath_img)

In [ ]:
!ls $path_rubincutout

In [ ]:
all_filenames = [ f"cutout_wcs_{key}_{band}.fits" for band in all_bands]
all_fullfilenames = [ os.path.join(path_rubincutout, f"cutout_wcs_{key}_{band}.fits") for band in all_bands] 

## Open cutout files, get wcs and normalize the images

In [ ]:
all_data_wcs_norm = []
for ib in range(NB):
    all_data_wcs_norm.append(load_fits_image_with_norm(all_fullfilenames[ib])) 

In [ ]:
def show_fits_grid(all_data_wcs_norm, ncols=3):
    NB = len(all_data_wcs_norm)
    nrows = (NB + ncols - 1) // ncols  # pour gérer les cas non multiples
    fig, axes = plt.subplots(nrows, ncols, figsize=(9 * ncols, 9 * nrows),
                             subplot_kw={'projection': all_data_wcs_norm[0][1]})

    # Flatten si axes est 2D
    axes = axes.flatten()

    for i in range(NB):
        band = all_bands[i]
        data, wcs, norm = all_data_wcs_norm[i]
        ax = axes[i]
        ax.imshow(data, origin='lower', cmap=all_bands_cmaps[i], norm=norm)
        #ax.imshow(data, origin='lower', cmap="grey", norm=norm)
        ax.set_xlabel("RA")
        ax.set_ylabel("Dec")
        ax.set_title(f"band {band}",fontweight="bold")

    # Masquer les axes restants si NB < nrows * ncols
    for j in range(NB, nrows * ncols):
        fig.delaxes(axes[j])

    plt.suptitle(target_title,fontweight="bold",fontsize=30)
    plt.tight_layout()
    plt.show()


In [ ]:
show_fits_grid(all_data_wcs_norm, ncols=3)

## Lupton

In [ ]:
combos = list(combinations(['g', 'r', 'i', 'z', 'y'], 3))

In [ ]:
combos

In [ ]:
def show_color_difference_combinations(band_data, bands=('g', 'r', 'i', 'z', 'y'),
                                       combos=None, ncols=3, cmap='seismic', stretch='linear'):
    """
    Affiche une grille des différences de magnitudes entre deux bandes (par ex. g - r, r - i, etc.)
    
    band_data : dict {band: data_array}
    combos : liste de tuples (band1, band2), ou None pour toutes les paires (band1 < band2)
    cmap : colormap à utiliser
    stretch : type de normalisation ('linear', 'asinh', 'log', etc.)
    """
    if combos is None:
        combos = list(combinations(bands, 2))  # toutes les paires ordonnées

    N = len(combos)
    nrows = (N + ncols - 1) // ncols
    fig, axes = plt.subplots(nrows, ncols, figsize=(5 * ncols, 5 * nrows))
    axes = axes.flatten()

    for i, (b1, b2) in enumerate(combos):
        ax = axes[i]
        try:
            diff = compute_color_difference(band_data[b1], band_data[b2])
            norm = simple_norm(diff[np.isfinite(diff)], stretch=stretch)  # exclut les NaN pour l'étendue
            im = ax.imshow(diff, origin='lower', cmap=cmap, norm=norm)
            ax.set_title(f"{b1} - {b2}", fontweight='bold')
            ax.axis('off')
            fig.colorbar(im, ax=ax, shrink=0.7)
        except Exception as e:
            ax.text(0.5, 0.5, f'Erreur\n{b1}-{b2}', ha='center', va='center', fontsize=10)
            print(f"Erreur pour {b1}-{b2} : {e}")
            ax.axis('off')

    for j in range(N, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.show()


### Prepare the data

In [ ]:
# Tu as déjà all_data_wcs_norm et band_names = ['u', 'g', 'r', 'i', 'z', 'y']
band_names = ['u', 'g', 'r', 'i', 'z', 'y']
band_data = {band_names[i]: all_data_wcs_norm[i][0] for i in range(len(band_names)) if band_names[i] != 'u'}


### Plot selection of combinations

In [ ]:
# Exemple simple avec les couleurs classiques
color_combos = [('g', 'r'), ('r', 'i'), ('i', 'z'), ('z', 'y'), ('g', 'z'), ('r', 'y')]

# Ou pour tester automatiquement toutes les combinaisons :
# color_combos = list(combinations(['g', 'r', 'i', 'z', 'y'], 2))

show_color_difference_combinations(band_data, combos=color_combos)

### Plot all combinations

In [ ]:
color_combos = list(combinations(['g', 'r', 'i', 'z', 'y'], 2))

In [ ]:
color_combos

In [ ]:
show_color_difference_combinations(band_data, combos=color_combos)

## Autre normalisation avec sigmaclip

In [ ]:
def show_color_difference_combinations_newnorm(band_data, bands=('g', 'r', 'i', 'z', 'y'),
                                       combos=None, ncols=3, cmap='seismic', stretch='linear'):
    """
    Affiche une grille des différences de magnitudes entre deux bandes (par ex. g - r, r - i, etc.)
    
    band_data : dict {band: data_array}
    combos : liste de tuples (band1, band2), ou None pour toutes les paires (band1 < band2)
    cmap : colormap à utiliser
    stretch : type de normalisation ('linear', 'asinh', 'log', etc.)
    """
    if combos is None:
        combos = list(combinations(bands, 2))  # toutes les paires ordonnées

    N = len(combos)
    nrows = (N + ncols - 1) // ncols
    fig, axes = plt.subplots(nrows, ncols, figsize=(5 * ncols, 5 * nrows))
    axes = axes.flatten()

    for i, (b1, b2) in enumerate(combos):
        ax = axes[i]
        try:
            diff_map = compute_color_difference(band_data[b1], band_data[b2])
            common_vmin,common_vmax = compute_common_limits(diff_map, quantile_clip=0.01)
            #norm = simple_norm(diff[np.isfinite(diff)], stretch=stretch)  # exclut les NaN pour l'étendue
            norm = simple_norm(diff_map, stretch='linear', vmin=common_vmin, vmax=common_vmax)
            im = ax.imshow(diff_map, origin='lower', cmap=cmap, norm=norm)
            ax.set_title(f"{b1} - {b2}", fontweight='bold')
            ax.axis('off')
            fig.colorbar(im, ax=ax, shrink=0.7)
        except Exception as e:
            ax.text(0.5, 0.5, f'Erreur\n{b1}-{b2}', ha='center', va='center', fontsize=10)
            print(f"Erreur pour {b1}-{b2} : {e}")
            ax.axis('off')

    for j in range(N, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.show()


In [ ]:
# Exemple simple avec les couleurs classiques
color_combos = [('g', 'r'), ('r', 'i'), ('i', 'z'), ('z', 'y'), ('g', 'z'), ('r', 'y')]

# Ou pour tester automatiquement toutes les combinaisons :
# color_combos = list(combinations(['g', 'r', 'i', 'z', 'y'], 2))

show_color_difference_combinations_newnorm(band_data, combos=color_combos)

In [ ]:
color_combos = list(combinations(['g', 'r', 'i', 'z', 'y'], 2))
show_color_difference_combinations_newnorm(band_data, combos=color_combos)